In [1]:
import cv2
import torch
import time
import numpy as np

In [14]:
pip install pymodbus==2.5.3


  Obtaining dependency information for pymodbus==2.5.3 from https://files.pythonhosted.org/packages/fb/0f/28190d06bbc3b78834d384cb04bb15e5dcdea4f5d370094dbf5debc3af10/pymodbus-2.5.3-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: pymodbus
    Found existing installation: pymodbus 3.7.4
    Uninstalling pymodbus-3.7.4:
      Successfully uninstalled pymodbus-3.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
midas 0.7.0 requires pymodbus<3.8.0,>=3.0.2; python_version >= "3.10", but you have pymodbus 2.5.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pymodbus
model_type = "DPT_Hybrid"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in /Users/jessie/.cache/torch/hub/intel-isl_MiDaS_master


ImportError: cannot import name 'IllegalFunctionRequest' from 'pymodbus.pdu' (/Users/jessie/anaconda3/lib/python3.11/site-packages/pymodbus/pdu/__init__.py)

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /Users/jessie/.cache/torch/hub/intel-isl_MiDaS_master


In [1]:
# Problem: the clips are too big. split them into scenes maybe

cap = cv2.VideoCapture('Desktop/UCLA/datares/downloaded_celebvhq/raw/_0tf2n3rlJU.mp4')

NameError: name 'cv2' is not defined

In [ ]:
while cap.isOpened():

    success, img = cap.read()

    start = time.time()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply input transforms
    input_batch = transform(img).to(device)

    # Prediction and resize to original resolution
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()

    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)


    end = time.time()
    totalTime = end - start

    fps = 1 / totalTime

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    depth_map = (depth_map*255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map , cv2.COLORMAP_MAGMA)

    # print('depth_map.shape = ',depth_map.shape)
    dim = (192*3, 108*4)
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    depth_map = cv2.resize(depth_map, dim, interpolation=cv2.INTER_AREA)

    # cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('Image', img)
    cv2.imshow('Depth Map', depth_map)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()